In [9]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import widgets
from IPython.display import display

In [10]:
allspecies = pd.read_csv("most_visited_nps_species_data.csv")
allspecies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61119 entries, 0 to 61118
Data columns (total 28 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ParkCode              61119 non-null  object 
 1   ParkName              61119 non-null  object 
 2   CategoryName          61119 non-null  object 
 3   Order                 60258 non-null  object 
 4   Family                60217 non-null  object 
 5   TaxonRecordStatus     61119 non-null  object 
 6   SciName               61119 non-null  object 
 7   CommonNames           34060 non-null  object 
 8   Synonyms              0 non-null      float64
 9   ParkAccepted          61119 non-null  bool   
 10  Sensitive             61119 non-null  bool   
 11  RecordStatus          61119 non-null  object 
 12  Occurrence            58666 non-null  object 
 13  OccurrenceTags        1445 non-null   object 
 14  Nativeness            56698 non-null  object 
 15  NativenessTags     

In [11]:
#Which parks have the most diverse categories(mammals, birds, reptiles, etc.)?
category_count_by_park = allspecies.groupby(['ParkName','CategoryName']).size().reset_index(name='Count')
category_count_by_park

,ParkName,CategoryName,Count
0,Acadia National Park,Amphibian,15
1,Acadia National Park,Bird,364
2,Acadia National Park,Fish,38
3,Acadia National Park,Mammal,55
4,Acadia National Park,Reptile,11
...,...,...,...
158,Zion National Park,Bird,301
159,Zion National Park,Fish,15
160,Zion National Park,Mammal,80
161,Zion National Park,Reptile,30


In [12]:
sortedCategories = sorted(allspecies['CategoryName'].unique())

fig_allparks = px.bar(category_count_by_park, 
                      x="ParkName", y="Count", color="CategoryName",
                      category_orders={'CategoryName': sortedCategories},
                      title="Species Categories Across Parks",
                      labels={'Count':'Number of Species'},
                      width=700, height=700)

fig_allparks.update_layout(xaxis_tickangle=-45,
                           legend_title_text="Species Cateogries")

fig_allparks.show()

In [13]:
#create dropdown widget for selecting Park
park_dropdown = widgets.Dropdown(
    description="Park Name: ",
    value="Yosemite National Park",
    options=category_count_by_park['ParkName'].unique().tolist()
)

#create bar chart for speices and their counts
def create_bar_chart(Category,Count):
    return[
        go.Bar(x=Category, y=Count, name='Num of Species', opacity=0.75)
    ]

#assign empty figure widget
g_bypark = go.FigureWidget(
    layout=go.Layout(
        title=dict(text="Species Categories by Yosemite National Park", x=0.5),
        barmode='group',
        xaxis_title='Species Category Name',
        yaxis_title='Number of Species',
        xaxis=dict(tickangle=-45),width=700, height=700
    )
)

def response(change):
    #filter df based on selected park
    selected_park = category_count_by_park[category_count_by_park['ParkName']==park_dropdown.value]

    #extract species category name and their corresponding counts 
    category = selected_park['CategoryName'].tolist()
    numOfspecies = selected_park['Count'].tolist()

    g_bypark.data = [] #clears existing data from figure to update with new data
    g_bypark.add_traces(create_bar_chart(category,numOfspecies)) #create new bar chart to the figure

    #update graph title based on user selection
    g_bypark.layout.title.text = f'Species Categories by {park_dropdown.value}'

park_dropdown.observe(response,names='value')
display(park_dropdown,g_bypark)
response(None)

Dropdown(description='Park Name: ', index=13, options=('Acadia National Park', 'Bryce Canyon National Park', '…

FigureWidget({
    'data': [],
    'layout': {'barmode': 'group',
               'height': 700,
               'template': '...',
               'title': {'text': 'Species Categories by Yosemite National Park', 'x': 0.5},
               'width': 700,
               'xaxis': {'tickangle': -45, 'title': {'text': 'Species Category Name'}},
               'yaxis': {'title': {'text': 'Number of Species'}}}
})

In [14]:
#create tab widget
tab = widgets.Tab()

eachpark_tab = widgets.VBox([park_dropdown, g_bypark])
allparks_tab = go.FigureWidget(fig_allparks)
tab.children=[allparks_tab,eachpark_tab]
tab.set_title(0, 'All Parks')
tab.set_title(1, 'Each Park')

park_dropdown.observe(response,names='value')
display(tab)
response(None)

    'data': [{'alignmentgroup': 'True',
              'hovertemplate': 'CategoryN…